In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Step 1: Install dependencies
!pip install transformers datasets evaluate huggingface_hub tqdm torch

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [24]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments, pipeline
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader, RandomSampler
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AdamW
from datasets import load_dataset
from tqdm import tqdm
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers.data.processors.squad import SquadProcessor
from transformers import squad_convert_examples_to_features
from transformers import AutoTokenizer
from tqdm.auto import tqdm
import evaluate
import collections
import numpy as np

In [25]:
from datasets import load_from_disk

# Load DatasetDict from Google Drive
dataset_dict_path = '/kaggle/input/news-qa-data/news-qa-data'
datasets = load_from_disk(dataset_dict_path)

# Verify the content
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'id'],
        num_rows: 57224
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'id'],
        num_rows: 14335
    })
})


In [26]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [27]:
max_length = 512
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]

        # Periksa jika jawaban ada
        if len(answer["answer_start"]) == 0 or len(answer["text"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [28]:
# mapping dataset tanpa caching
train_dataset = datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=datasets["train"].column_names,
    keep_in_memory=True 
)

len(datasets["train"]), len(train_dataset)

Map:   0%|          | 0/57224 [00:00<?, ? examples/s]

(57224, 99693)

In [29]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [30]:
validation_dataset = datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=datasets["validation"].column_names,
    keep_in_memory=True  
)
len(datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/14335 [00:00<?, ? examples/s]

(14335, 24856)

In [31]:
def compute_metrics(start_logits, end_logits, features, examples):
    metric = evaluate.load("squad")
    n_best = 20  
    max_answer_length = 200  
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": str(example_id), "prediction_text": ""})

    # Format references to match SQuAD evaluation
    theoretical_answers = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples]

    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [32]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [33]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-base-uncased-newsqa-squad-finetuned",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.916700
1000,1.930000
1500,1.676500
2000,1.591400
2500,1.526000
3000,1.443700
3500,1.438400
4000,1.460000
4500,1.393800
5000,1.342700


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=18693, training_loss=1.1274591832739236, metrics={'train_runtime': 17080.0529, 'train_samples_per_second': 17.51, 'train_steps_per_second': 1.094, 'total_flos': 7.814837270784614e+16, 'train_loss': 1.1274591832739236, 'epoch': 3.0})

In [35]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, datasets["validation"])

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


  0%|          | 0/14335 [00:00<?, ?it/s]

{'exact_match': 48.83850715033136, 'f1': 63.78600315037666}

In [19]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer, device='cuda')
context = """
AutoML: Automated Machine Learning tools like Auto-sklearn and H2O.ai simplify the process of building machine learning models.
"""
question = "What are some tools for automated machine learning?"
question_answerer(question=question, context=context)

{'score': 0.006397548597306013,
 'start': 47,
 'end': 68,
 'answer': 'Auto-sklearn and H2O.'}

In [36]:
from huggingface_hub import HfApi, HfFolder

api = HfApi()
token = "______________"
HfFolder.save_token(token)

In [37]:
model.push_to_hub("bert-base-uncased-newsqa-squad-finetuned")
tokenizer.push_to_hub("bert-base-uncased-newsqa-squad-finetuned")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Prasetyow12/bert-base-uncased-newsqa-squad-finetuned/commit/d01342d186ad8ad0f82423c248c8bc418a07dc8e', commit_message='Upload tokenizer', commit_description='', oid='d01342d186ad8ad0f82423c248c8bc418a07dc8e', pr_url=None, pr_revision=None, pr_num=None)

In [38]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load model dan tokenizer dari Hugging Face Hub
model_bert_qa = AutoModelForQuestionAnswering.from_pretrained("Prasetyow12/bert-base-uncased-newsqa-squad-finetuned")
tokenizer_bert_qa = AutoTokenizer.from_pretrained("Prasetyow12/bert-base-uncased-newsqa-squad-finetuned")

config.json:   0%|          | 0.00/673 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [39]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import re

# Buat pipeline untuk inference
question_answerer = pipeline("question-answering", model=model_bert_qa, tokenizer=tokenizer_bert_qa)

# Contoh konteks
context = """
(CNN) -- Police are investigating whether or what family issues might have prompted a California man to shoot six of his family members -- killing five -- before committing suicide. His wife was critically wounded. Authorities on Tuesday said Devan Kalathat, 42, shot his family Sunday night at his Santa Clara townhouse, killing two adults and three children. Kalathat killed his 11-year-old son, Akhil Dev; his 4-year-old daughter, Negha Dev; his 35-year-old brother-in-law Ashok Appu Poothemkandi; Poothemkandi's 25-year-old wife, Suchitra Sivaraman; and the couple's 11-month-old daughter, Ahana. Kalathat's 34-year-old wife, who was not identified, sustained multiple gunshot wounds and remains in critical condition, said Lt. Phil Cooke. "Family dynamics and personal relationships may have played a factor," Cooke told reporters Tuesday. He said Kalathat was employed as an engineer and nothing indicated he was facing "layoff or financial crisis." Investigators believe Kalathat used two .45-caliber semi-automatic pistols, both of which he owned. Cooke said Kalathat bought one of the pistols in February and the other nearly two weeks ago -- roughly the same time his wife's brother, Poothemkandi, arrived in California from India with Suchitra Sivaraman and Ahana. Cooke noted that Poothemkandi was an "educated professional" with plans to stay in the Bay Area to work on a project for a high-tech firm. Police were called after a neighbor noticed Kalathat's wounded wife outside the home around 8:30 p.m. (11:30 p.m. ET), Cooke said. When police arrived, other victims were found around the kitchen and dining room in what Cooke described as "a very gruesome scene." The family shooting comes just two months after a Los Angeles father who, after he and his wife were fired from their jobs, killed her and their five young children before turning the gun on himself.
"""
question = "What did police say was a factor?"

def clean_text(text):
    # Definisikan pola regex untuk berbagai cleaning
    url_pattern = re.compile(r'https?://\S+|www\.\S+', re.IGNORECASE)
    hashtag_pattern = re.compile(r'#\w+', re.IGNORECASE)
    double_space_pattern = re.compile(r'\s\s+')
    header_pattern = re.compile(r'^.*?--\s?', re.IGNORECASE)
    video_pattern = re.compile(r'VIDEO:.*?(?:\.\s|$)', re.IGNORECASE)

    # Hapus URL
    text = url_pattern.sub('', text)

    # Hapus hashtag
    text = hashtag_pattern.sub('', text)

    # Cek jika ada '--' dalam 40 karakter pertama
    if '--' in text[:40]:
        # Hapus header sebelum '--'
        text = header_pattern.sub('', text).strip()

    # Hapus frasa "VIDEO:" hingga titik
    text = video_pattern.sub('', text)

    # Hapus double space
    text = double_space_pattern.sub(' ', text)

    # Trim leading and trailing spaces
    text = text.strip()

    return text

# clean text
cleaned_context = clean_text(context)

# inferencing menggunakan data yang sudah clean
result = question_answerer(question=question, context=cleaned_context)

# Output hasil inference
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.9155185222625732, 'start': 744, 'end': 787, 'answer': '"Family dynamics and personal relationships'}


In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import re

# Load model dan tokenizer dari Hugging Face Hub
model_bert_qa = AutoModelForQuestionAnswering.from_pretrained("Prasetyow12/bert-base-uncased-newsqa-squad-finetuned")
tokenizer_bert_qa = AutoTokenizer.from_pretrained("Prasetyow12/bert-base-uncased-newsqa-squad-finetuned")

# Buat pipeline untuk inference
question_answerer = pipeline("question-answering", model=model_bert_qa, tokenizer=tokenizer_bert_qa)

# Contoh konteks
context = """
(CNN) -- Police are investigating whether or what family issues might have prompted a California man to shoot six of his family members -- killing five -- before committing suicide. His wife was critically wounded. Authorities on Tuesday said Devan Kalathat, 42, shot his family Sunday night at his Santa Clara townhouse, killing two adults and three children. Kalathat killed his 11-year-old son, Akhil Dev; his 4-year-old daughter, Negha Dev; his 35-year-old brother-in-law Ashok Appu Poothemkandi; Poothemkandi's 25-year-old wife, Suchitra Sivaraman; and the couple's 11-month-old daughter, Ahana. Kalathat's 34-year-old wife, who was not identified, sustained multiple gunshot wounds and remains in critical condition, said Lt. Phil Cooke. "Family dynamics and personal relationships may have played a factor," Cooke told reporters Tuesday. He said Kalathat was employed as an engineer and nothing indicated he was facing "layoff or financial crisis." Investigators believe Kalathat used two .45-caliber semi-automatic pistols, both of which he owned. Cooke said Kalathat bought one of the pistols in February and the other nearly two weeks ago -- roughly the same time his wife's brother, Poothemkandi, arrived in California from India with Suchitra Sivaraman and Ahana. Cooke noted that Poothemkandi was an "educated professional" with plans to stay in the Bay Area to work on a project for a high-tech firm. Police were called after a neighbor noticed Kalathat's wounded wife outside the home around 8:30 p.m. (11:30 p.m. ET), Cooke said. When police arrived, other victims were found around the kitchen and dining room in what Cooke described as "a very gruesome scene." The family shooting comes just two months after a Los Angeles father who, after he and his wife were fired from their jobs, killed her and their five young children before turning the gun on himself.
"""
question = "What did police say was a factor?"

# Fungsi untuk cleaning teks
def clean_text(text):
    # Definisikan pola regex untuk berbagai cleaning
    url_pattern = re.compile(r'https?://\S+|www\.\S+', re.IGNORECASE)
    hashtag_pattern = re.compile(r'#\w+', re.IGNORECASE)
    double_space_pattern = re.compile(r'\s\s+')
    header_pattern = re.compile(r'^.*?--\s?', re.IGNORECASE)
    video_pattern = re.compile(r'VIDEO:.*?(?:\.\s|$)', re.IGNORECASE)

    # Hapus URL
    text = url_pattern.sub('', text)

    # Hapus hashtag
    text = hashtag_pattern.sub('', text)

    # Cek jika ada '--' dalam 40 karakter pertama
    if '--' in text[:40]:
        # Hapus header sebelum '--'
        text = header_pattern.sub('', text).strip()

    # Hapus frasa "VIDEO:" hingga titik
    text = video_pattern.sub('', text)

    # Hapus double space
    text = double_space_pattern.sub(' ', text)

    # Trim leading and trailing spaces
    text = text.strip()

    return text

# Gunakan clean_text untuk cleaning konteks
cleaned_context = clean_text(context)

# Lakukan inference menggunakan konteks yang sudah dicleaning
result = question_answerer(question=question, context=cleaned_context)

# Output hasil inference hanya bagian jawabannya, tanpa tanda kutip
cleaned_answer = result['answer'].strip('"')

# Cetak jawaban yang sudah dibersihkan
print(cleaned_answer)


C:\Users\hp\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\hp\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Family dynamics and personal relationships
